# Effective Go
它为整个Go社区的Go代码提供了一个共同的基线。


在线运行go：
https://goplay.space/


In [ ]:
// 帮助函数
func Count(count *int) string {
    *count = *count + 1
    return fmt.Sprintf("%d:", *count)
}

# 函数

## 函数详情
1. 函数可以声明为类型
```go
type CalculateType func(int, int) // 声明了一个函数类型
```
2. 协程栈没有限制。Go语言中，函数还可以直接或间接地调用自己，也就是支持递归调用。Go语言函数的递归调用深度在逻辑上没有限制，函数调用的栈是不会出现溢出错误的，因为Go语言运行时会根据需要动态地调整函数栈的大小。每个Goroutine刚启动时只会分配很小的栈（4KB或8KB，具体依赖实现），根据需要动态调整栈的大小，栈最大可以达到GB级（依赖具体实现，在目前的实现中，32位体系结构为250MB，64位体系结构为1GB）。——《GO语言高级编程》

In [ ]:
// 函数的多个参数
func testArg(name string, more ... string) {
    fmt.Println(name)
    for _, v := range more {
        fmt.Println(v)
    }
}

func main() {
    testArg("hello", "a", "b", "c")
}

In [ ]:
// 闭包对捕获的外部变量并不是以传值方式访问，而是以引用方式访问。——《GO语言高级编程》

func Inc() (v int) {
    defer func(){ v++ } ()
    return 42
}

func Patch0() {
    for i := 0; i < 3; i++ {
        defer func(){ println(i) } ()
    }
}

func Patch1() {
    for i := 0; i < 3; i++ {
        defer func(i int){ println(i) } (i)
    }
}

func Patch2() {
    for i := 0; i < 3; i++ {
        i := i
        defer func(){ println(i) } ()
    }
}


// 闭包传递是引用。
func main() {
    fmt.Println(Inc())
    
    Patch0()
    Patch1()
    Patch2()
}

# 数据

## new 和 make

1. make 仅仅用于 slice，map，chan

In [ ]:
import "fmt"

func main () {
    slice0 := make([]int, 10, 100)
    fmt.Printf("len:%d cap:%d %v\n", len(slice0), cap(slice0), slice0)
    
    slice1 := make([]int, 10)
    fmt.Printf("len:%d cap:%d %v\n", len(slice1), cap(slice1), slice1)
    
    var invalidSlice []int
    invalidSlice = append(invalidSlice, 1)
    fmt.Println(invalidSlice)
}

## array & slice & map

In [ ]:
// 当触及到容量上限，append 将会返回新的数组
// https://stackoverflow.com/questions/41668053/cap-vs-len-of-slice-in-golang
func main() {
    s := make([]int, 0, 3)
    for i := 0; i < 5; i++ {
        s = append(s, i)
        fmt.Printf("cap %v, len %v, %p\n", cap(s), len(s), s)
    }
}

In [ ]:
//---------------------------------------------------
//
// array, slice, map使用复合字面
//
//---------------------------------------------------


import "fmt"

func main() {
    // key 是常量
    elements := []string{
        0:     "zero",
        1:     "one",
        4 / 2: "two",
    }
    fmt.Println(elements)
    fmt.Println("--------------------------")

    // index 不可以重复
    // elements = []string{
    //     0:     "zero",
    //     1:     "one",
    //     4 / 2: "two",
    //     2:     "also two",
    // }
    // fmt.Println(elements)

    // 跳过索引赋值
    numbers := []string{
        "a", "b", 2 << 1: "c", "d",
    }
    typeOfNumbers := reflect.TypeOf(numbers)
    fmt.Printf("name: %s, kind: %s\n", typeOfNumbers.Name(), typeOfNumbers.Kind())
    fmt.Printf("%#v\n", numbers)
    fmt.Println("--------------------------")
    
    // 指定size，将会是数组赋值
    stringArr := [3]string{"foo", "bar"}
    typeOfStringArr := reflect.TypeOf(stringArr)
    fmt.Printf("name: %s, kind: %s\n", typeOfStringArr.Name(), typeOfStringArr.Kind())
    fmt.Printf("%v\n", typeOfStringArr)
    fmt.Println("--------------------------")
    
    // 省略指定size，用 ... 代替。将会是数组赋值, ... 用最大的 index 自动计算
    stringArr1 := [...]string{"foo", "bar", 5: "test"}
    typeOfStringArr1 := reflect.TypeOf(stringArr1)
    fmt.Printf("name: %s, kind: %s\n", typeOfStringArr1.Name(), typeOfStringArr1.Kind())
    fmt.Printf("%v\n", typeOfStringArr1)
    fmt.Println("--------------------------")
    
    // map 的 key 不一定是字符串
    coords := map[[2]byte]string{{1, 1}: "one one", {2, 1}: "two one"}
    fmt.Printf("%v\n", coords)
    type Engineer struct {
        name string
        age  byte
    }
    engineers := [...]Engineer{{"Michał", 29}, {"John", 25}}    
    fmt.Printf("%v\n", engineers)
    fmt.Println("--------------------------")
}


In [ ]:
// copy 切片
// 1. 3:4 意味着1个元素，依然是左闭右开
func main() {
    
    count := 0

    
    names := [...]string{"张三","李四","王五","赵六","钱七"}
    nameSlice := names[3:4]
    fmt.Printf("%s %v\n", Count(&count), nameSlice)
    nameSlice2 := names[3:]
    fmt.Printf("%s %v\n", Count(&count), nameSlice2)
    
    fmt.Println("\n# copy 函数: func copy(dst, src []Type) int")
    slice1 := []int{1, 2, 3, 4, 5}
    slice2 := []int{5, 4, 3}
    copy(slice1, slice2)
    fmt.Println(Count(&count), slice2)

    strArr1 := [...]string{"张三","李四","王五","赵六","钱七"}
    strSlices1 := []string{"张三","李四","王五","赵六","钱七"}
    strSlices2 := []string{"amy", "jack"}
    copy(strSlices1, strSlices2)
    fmt.Printf("%s %v, %v\n", Count(&count), len(strSlices1), strSlices1)
    
    // 拷贝时如果 len(strSlices3) 为 0。不会分配新的数据
    // strSlices3 := make([]string, 0, 100)
    strSlices3 := make([]string, 3, 100)
    copy(strSlices3, strArr1[1:]) // 拷贝
    fmt.Printf("%s %v, %v\n", Count(&count), len(strSlices3), strSlices3)

    fmt.Println("\n# 删除切片元素")
    // 删除切片元素：尾部
    testArr1 := []int{1 ,2, 3}
    a2 := testArr1[:len(testArr1) - 1]
    a3 := testArr1[:len(testArr1)] // 切片尾部是最大index+1，也就是长度
    fmt.Println(Count(&count), a2, a3)
    
    // 删除切片元素：头部
    a4 := testArr1[0:] // 0 可以省略了
    a5 := testArr1[:]
    a6 := testArr1[1:]
    fmt.Println(Count(&count), a4, a5, a6)
    
    // 删除切片元素：头部（copy方式）。删除头部两个
    testArr3 := []int{1 ,2, 3}
    a9 := testArr3[:copy(testArr3[:], testArr3[2:])]
    fmt.Println(Count(&count), a9)
    
    // 删除中间元素 2
    testArr2 := []int{1 ,2, 3}
    fmt.Println(Count(&count), testArr2)
    a10 := append(testArr2[:1], testArr2[2:]...)
    fmt.Println(Count(&count), a10)
    testArr5 := []int{1 ,2, 3}
    fmt.Println(Count(&count), testArr5)
    testArr5 = testArr5[:1 + copy(testArr5[1:], testArr5[2:])] // 1 是中间的中位数 index
    fmt.Println(Count(&count), testArr5)
    
    fmt.Println("\n# a7的地址 和 a7值指向的地址都没有改变")
    testArr4 := []int{1 ,2, 3}
    a7 := testArr4
    fmt.Printf("%s %p %p\n", Count(&count), a7, &a7)
    a7 = append(a7[:0], testArr4[1:]...)
    fmt.Printf("%s %p %p\n", Count(&count), a7, &a7)
    fmt.Println(Count(&count), a7)
}

## 字符串

In [ ]:
// 字符串支持切片
func main() {
    s := "hello world"
    fmt.Println(s[2:])
}

In [ ]:
// 字符串转义字符、中文字符切片
func main() {
    fmt.Println("\xe4\xb8\x96")
    fmt.Println("\xe7\x95\x8c")
    s := "\xe4\xb8\x96\xe7\x95\x8c"
    
    fmt.Println(s)
    
    // 转义字符切片没有按照转义的中文字符数
    fmt.Println(s[1:])
    
    // 转义字符 for range 没有自动解码。但是迭代次数是对的。
    for k, v := range "\xe4\xb8\x96\xe7\x95\x8c" {
        fmt.Printf("%d %s\n", k, v)
    }
    
    // 这样就是单个显示。int32 每个最大4个字节
    for i:= 0; i < len(s); i++ {
        fmt.Printf("%d %s\n", i, s[i])
    }
    
    // 非转义的字符切片同样没有中文字符数
    s2 := "你好世界"
    fmt.Println(s2)
    fmt.Println(s2[1:])
    
    // 字符串是不可变的。能赋值吗?
    // s2[1] = byte('H')
    // fmt.Println(s2)
}


In [ ]:
// rune转字符串
func main() {
    fmt.Printf("%#v\n", []rune{'世', '界'})
    fmt.Printf("%#v\n", []rune("世界"))
    fmt.Printf("%#v\n", string([]rune("世界")))
}

In [ ]:
// 字符串格式化
func main() {
    // 通用
    fmt.Printf("%v\n", 100)
    fmt.Printf("%v\n", false)
    o := struct{ name string }{"小王子"}
    fmt.Printf("%v\n", o)
    fmt.Printf("%#v\n", o)
    fmt.Printf("%T\n", o)
    fmt.Printf("100%%\n")
    fmt.Printf("\n\n")
    
    // 布尔
    fmt.Printf("%t\n", true)
    fmt.Printf("%t\n", false)
    fmt.Printf("\n\n")
    
    // 整数：二进制 八进制 十进制 十六进制
    fmt.Printf("整数\n")
    n := 65
    fmt.Printf("%b\n", n)
    fmt.Printf("%c\n", n)
    fmt.Printf("%d\n", n)
    fmt.Printf("%o\n", n)
    fmt.Printf("%x\n", n)
    fmt.Printf("%X\n", n)
    fmt.Printf("\n\n")
    
    fmt.Printf("浮点数\n")
    // 浮点数
    f := 12.34
    fmt.Printf("%b\n", f)
    fmt.Printf("%e\n", f)
    fmt.Printf("%E\n", f)
    fmt.Printf("%f\n", f)
    fmt.Printf("%g\n", f)
    fmt.Printf("%G\n", f)
    
    f = 12.34567
    fmt.Printf("%f\n", f)     // 默认宽度，默认精度
    fmt.Printf("%9f\n", f)    // 宽度9，默认精度
    fmt.Printf("%.2f\n", f)   // 默认宽度，精度2
    fmt.Printf("%9.2f\n", f)  // 宽度9，精度2
    fmt.Printf("%9.f\n", f)   // 宽度9，精度0
    fmt.Printf("\n\n")
    
    fmt.Printf("字符串\n")
    // 字符串
    s := "小王子"
    fmt.Printf("%s\n", s) // 直接输出字符串或者[]byte
    fmt.Printf("%q\n", s) // 该值对应的双引号括起来的go语法字符串字面值，必要时会采用安全的转义表示
    fmt.Printf("%x\n", s) // 每个字节用两字符十六进制数表示（使用a-f)
    fmt.Printf("%X\n", s) // 每个字节用两字符十六进制数表示（使用A-F）
    fmt.Printf("\n\n")
    
    fmt.Printf("指针\n")
    // 指针
    a := 10
    fmt.Printf("%p\n", &a)
    fmt.Printf("%#p\n", &a)
    
}

# 并发

In [256]:
import (
    "sync"
)

type Total struct {
    sync.Mutex
    value int
}

func worker(total *Total, wg *sync.WaitGroup) {
    defer wg.Done()
    for i := 0; i < 100; i++ {
        total.Lock()
        total.value += 1
        total.Unlock()
    }
}

func main() {
    var wg sync.WaitGroup
    var total Total
    wg.Add(2)
    go worker(&total, &wg)
    go worker(&total, &wg)
    wg.Wait()
    fmt.Println(total.value)
}

200


In [255]:
import (
    "sync"
    "sync/atomic"
)

type Total struct {
    sync.Mutex
    value int
}

func main() {
    
    var total uint64
    worker := func (wg *sync.WaitGroup) {
        defer wg.Done()
        for i := 0; i < 100; i++ {
            atomic.AddUint64(&total, 1)
        }
    }
    var wg sync.WaitGroup
    wg.Add(2)
    go worker(&wg)
    go worker(&wg)
    wg.Wait()
    
    fmt.Println(total)
}

200
